In [1]:
import os
import csv
from towhee import ops, pipe, register
from towhee.operator import PyOperator
from towhee import DataCollection
from tqdm import tqdm
import pandas as pd
import json
import numpy as np
from helpers import milvus_utils

Connected to Milvus server at port 19530


In [121]:
# CONSTANTS

# Files
MSRVTT_SAMPLES = "./MSRVTT_1K.csv"
FIRE_BENCHMARK_Q_JUDGEMENTS = "./fire_benchmark_q_judgements.csv" # file created using raw FIRE judgements

# Database Collections
VIDEO_RET_COLLECTION = "msrvtt_vid_ret_1"
FRAME_RET_COLLECTION = "msrvtt_frame_ret_1"


In [4]:

raw_samples_df = pd.read_csv(MSRVTT_SAMPLES)
raw_samples_df[['video_id', 'video_path', 'sentence']].head()

,video_id,video_path,sentence
0,video7579,./test_1k_compress/video7579.mp4,a girl wearing red top and black trouser is pu...
1,video7725,./test_1k_compress/video7725.mp4,young people sit around the edges of a room cl...
2,video9258,./test_1k_compress/video9258.mp4,a person is using a phone
3,video7365,./test_1k_compress/video7365.mp4,cartoon people are eating at a restaurant
4,video8068,./test_1k_compress/video8068.mp4,a woman on a couch talks to a a man


In [8]:
milvus_utils.create_milvus_collection(VIDEO_RET_COLLECTION, 512)

<Collection>:
-------------
<name>: msrvtt_vid_ret_1
<description>: video retrieval
<schema>: {'auto_id': False, 'description': 'video retrieval', 'fields': [{'name': 'id', 'description': '', 'type': <DataType.INT64: 5>, 'is_primary': True, 'auto_id': False}, {'name': 'embedding', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 512}}]}

We create a pipeline that loads the video embeddings into the Milvus Vector DB using a distributed Twohee pipeline

In [ ]:
def read_loader_csv(csv_file):
    with open(csv_file, 'r', encoding='utf-8-sig') as f:
        data = csv.DictReader(f)
        for line in data:
            yield int(line['video_id'][len('video'):]), line['video_path']

video_loader_pipeline = (
    pipe.input('csv_file')
    .flat_map('csv_file', ('video_id', 'video_path'), read_loader_csv)
    # Create 12 evenly distributed frames per video
    .map('video_path', 'frames', ops.video_decode.ffmpeg(sample_type='uniform_temporal_subsample', args={'num_samples': 12}))
    # I have a M2 Max, so device is set to mps for better performance
    .map('frames', 'vec', ops.video_text_embedding.clip4clip(model_name='clip_vit_b32', modality='video', device='mps'))
    .map(('video_id', 'vec'), (), ops.ann_insert.milvus_client(collection_name=VIDEO_RET_COLLECTION))
    .output('video_id')
)

In [11]:
# We call the pipeline with a CSV file containing the video paths
video_loader_ret = video_loader_pipeline(MSRVTT_SAMPLES)

2025-04-16 15:04:15,510 - 17173606400 - node.py-node:167 - INFO: Begin to run Node-_input
2025-04-16 15:04:15,510 - 17190432768 - node.py-node:167 - INFO: Begin to run Node-read_loader_csv-0
2025-04-16 15:04:15,511 - 17207259136 - node.py-node:167 - INFO: Begin to run Node-video-decode/ffmpeg-1
2025-04-16 15:04:15,511 - 17224085504 - node.py-node:167 - INFO: Begin to run Node-video-text-embedding/clip4clip-2
2025-04-16 15:04:15,512 - 17240911872 - node.py-node:167 - INFO: Begin to run Node-ann-insert/milvus-client-3
2025-04-16 15:04:15,512 - 17173606400 - node.py-node:167 - INFO: Begin to run Node-_output


The 1000 videos are now loaded into the Milvus `VIDEO_RET_COLLECTION` collection.

Now, we query these videos using the annotated sentences as queries and the video ids as the ground truth results.

In [25]:
def read_video_search_csv(csv_file):
    import csv
    with open(csv_file, 'r', encoding='utf-8-sig') as f:
        data = csv.DictReader(f)
        for line in data:
            yield line['video_id'], line['sentence']

video_search_pipeline = (
    pipe.input('csv_file')
    .flat_map('csv_file', ('rel_video_id', 'query'), read_video_search_csv)
    .map('query', 'vec', ops.video_text_embedding.clip4clip(model_name='clip_vit_b32', modality='text', device='mps'))
    .map('vec', 'top10_raw_res', 
         ops.ann_search.milvus_client(collection_name=VIDEO_RET_COLLECTION, limit=10))
    .map('top10_raw_res', ('top1', 'top5', 'top10'), lambda x: (x[:1], x[:5], x[:10]))
    .output('rel_video_id', 'query', 'top1', 'top5', 'top10')
)

In [ ]:
all_query_results = DataCollection(video_search_pipeline(MSRVTT_SAMPLES))

2025-04-16 15:12:12,149 - 17800654848 - node.py-node:167 - INFO: Begin to run Node-_input
2025-04-16 15:12:12,150 - 17842597888 - node.py-node:167 - INFO: Begin to run Node-read_video_search_csv-0
2025-04-16 15:12:12,150 - 17976815616 - node.py-node:167 - INFO: Begin to run Node-video-text-embedding/clip4clip-1
2025-04-16 15:12:12,150 - 19253981184 - node.py-node:167 - INFO: Begin to run Node-ann-search/milvus-client-2
2025-04-16 15:12:12,150 - 19273904128 - node.py-node:167 - INFO: Begin to run Node-lambda-3
2025-04-16 15:12:12,151 - 21981327360 - node.py-node:167 - INFO: Begin to run Node-_output


2025-04-16 16:26:41,865 - 17800654848 - node.py-node:167 - INFO: Begin to run Node-_input
2025-04-16 16:26:41,865 - 17842597888 - node.py-node:167 - INFO: Begin to run Node-read_video_search_csv-0
2025-04-16 16:26:41,865 - 17976815616 - node.py-node:167 - INFO: Begin to run Node-video-text-embedding/clip4clip-1
2025-04-16 16:26:41,865 - 19253981184 - node.py-node:167 - INFO: Begin to run Node-ann-search/milvus-client-2
2025-04-16 16:26:41,865 - 19273904128 - node.py-node:167 - INFO: Begin to run Node-lambda-3
2025-04-16 16:26:41,865 - 21981327360 - node.py-node:167 - INFO: Begin to run Node-_output
2025-04-16 16:26:41,869 - 17842597888 - node.py-node:142 - INFO: read_video_search_csv-0 ends with status: NodeStatus.FAILED
2025-04-16 16:27:03,120 - 17800654848 - node.py-node:167 - INFO: Begin to run Node-_input
2025-04-16 16:27:03,120 - 17842597888 - node.py-node:167 - INFO: Begin to run Node-read_video_search_csv-0
2025-04-16 16:27:03,120 - 17976815616 - node.py-node:167 - INFO: Begin t

In [31]:
all_query_results.show()

rel_video_id,query,top1,top5,top10
video7579,a girl wearing red top and black trouser is putting a sweater on a dog,"[[7579, 1.415151834487915]] len=1","[[7579, 1.415151834487915],[9969, 1.4799106121063232],[8837, 1.4897732734680176],[9347, 1.4948583841323853],...] len=5","[[7579, 1.415151834487915],[9969, 1.4799106121063232],[8837, 1.4897732734680176],[9347, 1.4948583841323853],...] len=10"
video7725,young people sit around the edges of a room clapping and raising their arms while others dance in the center during a party,"[[7725, 1.3622068166732788]] len=1","[[7725, 1.3622068166732788],[8014, 1.4865269660949707],[8339, 1.4922078847885132],[8442, 1.5024112462997437],...] len=5","[[7725, 1.3622068166732788],[8014, 1.4865269660949707],[8339, 1.4922078847885132],[8442, 1.5024112462997437],...] len=10"
video9258,a person is using a phone,"[[9258, 1.4011969566345215]] len=1","[[9258, 1.4011969566345215],[9257, 1.422863483428955],[9697, 1.4413853883743286],[7910, 1.4945621490478516],...] len=5","[[9258, 1.4011969566345215],[9257, 1.422863483428955],[9697, 1.4413853883743286],[7910, 1.4945621490478516],...] len=10"
video7365,cartoon people are eating at a restaurant,"[[7365, 1.4027695655822754]] len=1","[[7365, 1.4027695655822754],[8781, 1.4623048305511475],[9537, 1.4739768505096436],[7831, 1.505112648010254],...] len=5","[[7365, 1.4027695655822754],[8781, 1.4623048305511475],[9537, 1.4739768505096436],[7831, 1.505112648010254],...] len=10"
video8068,a woman on a couch talks to a a man,"[[7162, 1.4716743230819702]] len=1","[[7162, 1.4716743230819702],[8304, 1.4787472486495972],[8068, 1.4926888942718506],[7724, 1.4982554912567139],...] len=5","[[7162, 1.4716743230819702],[8304, 1.4787472486495972],[8068, 1.4926888942718506],[7724, 1.4982554912567139],...] len=10"


In [ ]:
# Convert the Twohee data collection to a pandas dataframe so we can apply evaluation methods

In [131]:
# TODO add this entire thing into its own module

def twohee_data_col_to_df(twohee_data_collection):
    res_list = twohee_data_collection.to_list()
    res_obj_list = []
    for r in res_list:
        res_obj = vars(r)
        res_obj_list.append(res_obj)
    res_df = pd.DataFrame(res_obj_list)
    # Add ground truth column
    res_df['ground_truth'] = res_df['rel_video_id'].apply(lambda x: int(x[len('video'):]))
    return res_df.copy()


def average_precision(ground_truth, predictions):
    """
    Calculate the Average Precision (AP) for a single query.

    Args:
        ground_truth (int): The ground truth video ID.
        predictions (list): List of predicted video IDs.

    Returns:
        float: The Average Precision (AP) score for the query.
    """
    hits = 0
    sum_precision = 0
    for i, pred in enumerate(predictions):
        if pred == ground_truth:
            hits += 1
            sum_precision += hits / (i + 1)
    return sum_precision / hits if hits > 0 else 0


def calculate_mean_average_precision(df):
    """
    Calculate the Mean Average Precision (MAP) for the given dataframe.

    Args:
        df (pd.DataFrame): DataFrame containing columns 'query', 'ground_truth', 'top1', 'top5', 'top10'.

    Returns:
        float: The Mean Average Precision (MAP) score.
    """
    # Calculate AP for each query
    ap_scores = []
    for _, row in df.iterrows():
        ground_truth = row['ground_truth']
        predictions_with_scores = row['top10']
        predictions = [pred[0] for pred in predictions_with_scores]
        ap_scores.append(average_precision(ground_truth, predictions))

    # Calculate MAP
    mean_ap = sum(ap_scores) / len(ap_scores) if ap_scores else 0
    return mean_ap


def calculate_recall(df):
    """
    Calculate recall@1, recall@5, and recall@10 for the given dataframe.

    Args:
        df (pd.DataFrame): DataFrame containing columns 'query', 'ground_truth', 'top1', 'top5', 'top10'.

    Returns:
        dict: A dictionary containing recall@1, recall@5, and recall@10.
    """
    recall_at_1 = 0
    recall_at_5 = 0
    recall_at_10 = 0
    total_queries = len(df)

    for _, row in df.iterrows():
        ground_truth = row['ground_truth']
        if ground_truth in [pred[0] for pred in row['top1']]:
            recall_at_1 += 1
        if ground_truth in [pred[0] for pred in row['top5']]:
            recall_at_5 += 1
        if ground_truth in [pred[0] for pred in row['top10']]:
            recall_at_10 += 1

    return {
        'recall@1': recall_at_1 / total_queries,
        'recall@5': recall_at_5 / total_queries,
        'recall@10': recall_at_10 / total_queries
    }
    
def ndcg_score(ground_truth, predictions, k=10):
    """
    Calculate the Normalized Discounted Cumulative Gain (NDCG) for a single query.

    Args:
        ground_truth (int): The ground truth video ID.
        predictions (list): List of predicted video IDs with scores [(id, score), ...].
        k (int): The number of top predictions to consider.

    Returns:
        float: The NDCG score for the query.
    """
    def dcg(relevance_scores):
        return sum(rel / np.log2(idx + 2) for idx, rel in enumerate(relevance_scores))

    # Relevance scores: 1 if the prediction matches the ground truth, else 0
    relevance_scores = [1 if pred[0] == ground_truth else 0 for pred in predictions[:k]]

    # Calculate DCG and IDCG
    actual_dcg = dcg(relevance_scores)
    ideal_dcg = dcg(sorted(relevance_scores, reverse=True))

    # Return NDCG
    return actual_dcg / ideal_dcg if ideal_dcg > 0 else 0

# call this function to get the NDCG score for each query
def calculate_ndcg(df, k=10):
    """
    Calculate NDCG for the given dataframe.

    Args:
        df (pd.DataFrame): DataFrame containing columns 'query', 'ground_truth', 'top1', 'top5', 'top10'.
        k (int): The number of top predictions to consider.

    Returns:
        float: The mean NDCG score.
    """
    ndcg_scores = []
    for _, row in df.iterrows():
        ground_truth = row['ground_truth']
        predictions_with_scores = row['top10']
        ndcg_scores.append(ndcg_score(ground_truth, predictions_with_scores, k))

    return sum(ndcg_scores) / len(ndcg_scores) if ndcg_scores else 0

def get_all_eval_scores(df):
    """Return a dataframe with all evaluation scores: Recall@1, Recall@5, Recall@10, MAP, NDCG@1, NDCG@5, NDCG@10"""
    recall_scores = calculate_recall(df)
    map_score = calculate_mean_average_precision(df)
    ndcg_score_1 = calculate_ndcg(df, k=1)
    ndcg_score_5 = calculate_ndcg(df, k=5)
    ndcg_score_10 = calculate_ndcg(df, k=10)

    eval_scores = {
        'recall@1': recall_scores['recall@1'],
        'recall@5': recall_scores['recall@5'],
        'recall@10': recall_scores['recall@10'],
        'map': map_score,
        'ndcg@1': ndcg_score_1,
        'ndcg@5': ndcg_score_5,
        'ndcg@10': ndcg_score_10
    }
    
    return eval_scores

In [63]:
all_query_results_df = twohee_data_col_to_df(all_query_results)
# Create ground truth column as video id casted into an int, this is used for the eval functions
all_query_results_df['ground_truth'] = all_query_results_df['rel_video_id'].apply(lambda x: int(x[len('video'):]))
all_query_results_df

,rel_video_id,query,top1,top5,top10,ground_truth
0,video7579,a girl wearing red top and black trouser is pu...,"[[7579, 1.415151834487915]]","[[7579, 1.415151834487915], [9969, 1.479910612...","[[7579, 1.415151834487915], [9969, 1.479910612...",7579
1,video7725,young people sit around the edges of a room cl...,"[[7725, 1.3622068166732788]]","[[7725, 1.3622068166732788], [8014, 1.48652696...","[[7725, 1.3622068166732788], [8014, 1.48652696...",7725
2,video9258,a person is using a phone,"[[9258, 1.4011969566345215]]","[[9258, 1.4011969566345215], [9257, 1.42286348...","[[9258, 1.4011969566345215], [9257, 1.42286348...",9258
3,video7365,cartoon people are eating at a restaurant,"[[7365, 1.4027695655822754]]","[[7365, 1.4027695655822754], [8781, 1.46230483...","[[7365, 1.4027695655822754], [8781, 1.46230483...",7365
4,video8068,a woman on a couch talks to a a man,"[[7162, 1.4716743230819702]]","[[7162, 1.4716743230819702], [8304, 1.47874724...","[[7162, 1.4716743230819702], [8304, 1.47874724...",8068
...,...,...,...,...,...,...
995,video7034,man in black shirt is holding a baby upside do...,"[[9320, 1.5113091468811035]]","[[9320, 1.5113091468811035], [9404, 1.51643335...","[[9320, 1.5113091468811035], [9404, 1.51643335...",7034
996,video7568,the queen of england is seen walking with an e...,"[[7568, 1.2981326580047607]]","[[7568, 1.2981326580047607], [7116, 1.41021490...","[[7568, 1.2981326580047607], [7116, 1.41021490...",7568
997,video7979,people talking about a fight,"[[7211, 1.4528591632843018]]","[[7211, 1.4528591632843018], [7979, 1.46294164...","[[7211, 1.4528591632843018], [7979, 1.46294164...",7979
998,video7356,a vehicle with details on what comes with it b...,"[[7356, 1.4014551639556885]]","[[7356, 1.4014551639556885], [7765, 1.47221362...","[[7356, 1.4014551639556885], [7765, 1.47221362...",7356


In [68]:
recall_metrics = calculate_recall(all_query_results_df)
map_value = calculate_mean_average_precision(all_query_results_df)

print("Recall metrics at top-1, 5, 10:")
print(recall_metrics)
print("Mean Average Precision (MAP):")
print(map_value)

Recall metrics at top-1, 5, 10:
{'recall@1': 0.426, 'recall@5': 0.716, 'recall@10': 0.814}
Mean Average Precision (MAP):
0.5456543650793645


In [81]:
# Calculate NDCG with k=1, 5, 10
for k in [1, 5, 10]:
    ndcg_value = calculate_ndcg(all_query_results_df, k=k)
    print(f"NDCG@{k}: {ndcg_value}")


NDCG@1: 0.426
NDCG@5: 0.5780321865313451
NDCG@10: 0.6100154270801753


In [83]:
get_all_eval_scores(all_query_results_df)

{'recall@1': 0.426,
 'recall@5': 0.716,
 'recall@10': 0.814,
 'map': 0.5456543650793645,
 'ndcg@1': 0.426,
 'ndcg@5': 0.5780321865313451,
 'ndcg@10': 0.6100154270801753}

## Try evaluation against queries from FIRE benchmark

We are working with a sample of MSR-VTT and our evaluation pipeline supports only one relevant query per video, hence we need to filter the full FIRE benchmark to only include videos we have sampled and ones with a single relevant result.

In [86]:
# Load FIRE benchmark queries and judgements
FIRE_MSRVTT_RAW = "data/FIRE/fire-data/fire_msrvtt_dataset.json"



In [87]:
def open_json(path):
    with open(path) as f:
        data = json.load(f)
    return data

fire_data = open_json(FIRE_MSRVTT_RAW)
fire_judgements_raw_df = pd.DataFrame(fire_data["annotations"])
fire_judgements_raw_df

,annotator_ids,label,video_id,query,annotator_labels,response_ids,queue_ids,models,dataset
0,[84],irrelevant,video7026,a 3d animation of a cabinet with plates,[irrelevant],[514705627045591],[4985131491553502],[CE],MSRVTT
1,[62],irrelevant,video7418,a 3d animation of a cabinet with plates,[irrelevant],[1009913593254130],[4985131491553502],[CE],MSRVTT
2,[50],irrelevant,video7763,a 3d animation of a cabinet with plates,[irrelevant],[3280646372150702],[4995314980504271],[SSB],MSRVTT
3,[67],relevant,video7768,a 3d animation of a cabinet with plates,[relevant],[1014281349443178],[5083898558315210],[CLIP4CLIP],MSRVTT
4,[55],irrelevant,video7822,a 3d animation of a cabinet with plates,[irrelevant],[2619512781525857],[4995314980504271],[SSB],MSRVTT
...,...,...,...,...,...,...,...,...,...
24162,[69],irrelevant,video9770,young people sit around the edges of a room cl...,[irrelevant],[310721911172645],[4995314980504271],[SSB],MSRVTT
24163,[40],irrelevant,video9788,young people sit around the edges of a room cl...,[irrelevant],[5180951918610413],[4985131491553502],[CE],MSRVTT
24164,[36],irrelevant,video9814,young people sit around the edges of a room cl...,[irrelevant],[498969301872125],[4985131491553502],[CE],MSRVTT
24165,[57],irrelevant,video9815,young people sit around the edges of a room cl...,[irrelevant],[1154835725293034],[5083898558315210],[CLIP4CLIP],MSRVTT


In [ ]:
def get_cleaned_fire_judgements(raw_fire_annotations):
    all_queries = [a["query"] for a in raw_fire_annotations]
    print("Total queries", len(all_queries))
    unique_queries = list(set(all_queries))
    print("Unique queries", len(unique_queries))
    
    # Initialize lists to store relevant and non-relevant IDs
    relevant_ids = []
    non_relevant_ids = []
    
    # Iterate through the first queries and categorize the IDs
    for query in unique_queries:
        relevant = []
        non_relevant = []
        for annotation in raw_fire_annotations:
            if annotation["query"] == query:
                if annotation["label"] == "relevant":
                    relevant.append(annotation["video_id"])
                else:
                    non_relevant.append(annotation["video_id"])
        relevant_ids.append(relevant)
        non_relevant_ids.append(non_relevant)
        

    return pd.DataFrame({
    "query": unique_queries,
    "relevant_ids": relevant_ids,
    "non_relevant_ids": non_relevant_ids
    }).copy()
    

In [106]:
fire_judgements_df = get_cleaned_fire_judgements(fire_data["annotations"])
fire_judgements_df.info()

Total queries 24167
Unique queries 995
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 995 entries, 0 to 994
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   query             995 non-null    object
 1   relevant_ids      995 non-null    object
 2   non_relevant_ids  995 non-null    object
dtypes: object(3)
memory usage: 23.4+ KB


In [113]:
# Filter only queries with a single relevant result
fire_judgements_single_result = fire_judgements_df[fire_judgements_df["relevant_ids"].apply(lambda x: len(x) == 1)]
fire_judgements_single_result

,query,relevant_ids,non_relevant_ids
0,two parrots in a bird cage one white chick and...,[video8469],"[video7113, video7151, video7160, video7342, v..."
4,a man chopping lobster and taking off the shell,[video9687],"[video7147, video7462, video7559, video7699, v..."
5,two women are walking in a parking lot,[video7698],"[video7138, video7177, video7212, video7223, v..."
8,a woman is talking about how jeans with patche...,[video9503],"[video7233, video7544, video7549, video7572, v..."
9,a naked child runs through a field,[video8903],"[video7021, video7155, video7217, video7363, v..."
...,...,...,...
987,selena gomez clips of her videos and her danci...,[video9801],"[video7111, video7162, video7370, video7411, v..."
990,video of gymasts practicing to roll,[video8267],"[video7111, video7118, video7131, video7153, v..."
991,a woman walking along side a river in a bikini,[video8928],"[video7142, video7152, video7204, video7598, v..."
993,a little girl talking to her and is scared,[video7782],"[video7118, video7587, video7946, video8018, v..."


This is our final benchmark queries:

In [127]:
video_ids_in_sample = set(raw_samples_df["video_id"].apply(lambda x: int(x[len("video"):])))
final_fire_jud = fire_judgements_single_result[fire_judgements_single_result["relevant_ids"].apply(lambda x: int(x[0][len("video"):]) in video_ids_in_sample)]
final_fire_jud['video_id'] = final_fire_jud['relevant_ids'].apply(lambda x: x[0])
final_fire_jud['sentence'] = final_fire_jud['query']
final_fire_jud

,query,relevant_ids,non_relevant_ids,video_id,sentence
0,two parrots in a bird cage one white chick and...,[video8469],"[video7113, video7151, video7160, video7342, v...",video8469,two parrots in a bird cage one white chick and...
4,a man chopping lobster and taking off the shell,[video9687],"[video7147, video7462, video7559, video7699, v...",video9687,a man chopping lobster and taking off the shell
5,two women are walking in a parking lot,[video7698],"[video7138, video7177, video7212, video7223, v...",video7698,two women are walking in a parking lot
8,a woman is talking about how jeans with patche...,[video9503],"[video7233, video7544, video7549, video7572, v...",video9503,a woman is talking about how jeans with patche...
9,a naked child runs through a field,[video8903],"[video7021, video7155, video7217, video7363, v...",video8903,a naked child runs through a field
...,...,...,...,...,...
987,selena gomez clips of her videos and her danci...,[video9801],"[video7111, video7162, video7370, video7411, v...",video9801,selena gomez clips of her videos and her danci...
990,video of gymasts practicing to roll,[video8267],"[video7111, video7118, video7131, video7153, v...",video8267,video of gymasts practicing to roll
991,a woman walking along side a river in a bikini,[video8928],"[video7142, video7152, video7204, video7598, v...",video8928,a woman walking along side a river in a bikini
993,a little girl talking to her and is scared,[video7782],"[video7118, video7587, video7946, video8018, v...",video7782,a little girl talking to her and is scared


In [128]:
# Create csv to be used in query pipeline
final_fire_jud.to_csv(FIRE_BENCHMARK_Q_JUDGEMENTS)

In [129]:
# Run query pipeline using FIRE
fire_query_results = DataCollection(video_search_pipeline(FIRE_BENCHMARK_Q_JUDGEMENTS))

In [133]:
get_all_eval_scores(twohee_data_col_to_df(fire_query_results))

{'recall@1': 0.5732484076433121,
 'recall@5': 0.821656050955414,
 'recall@10': 0.9076433121019108,
 'map': 0.6800513092710544,
 'ndcg@1': 0.5732484076433121,
 'ndcg@5': 0.7064313418462581,
 'ndcg@10': 0.7348170561659407}